<a href="https://colab.research.google.com/github/JEx2VAN/MarketGap-miner-/blob/main/Project_market_miner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies #

In [ ]:
#  For data handling
!pip install pandas numpy

# For basic NLP and sentiment
!pip install spacy nltk vaderSentiment
!python -m spacy download en_core_web_sm

# For topic modeling
!pip install bertopic

# For advanced NLP models (used by BERTopic)
!pip install transformers

# For the dashboard and running it in Colab
!pip install streamlit pyngrok

#  Dependencies for browser automation (kept from original, though unused in this script)
!apt-get update
!apt-get install -y chromium-driver
!pip install selenium

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import os
import threading
import time

# NLP Libraries
import nltk
from nltk.corpus import stopwords
import spacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bertopic import BERTopic

# Dashboard Libraries
import streamlit as st
from pyngrok import ngrok

# Google Drive
from google.colab import drive

# Download NLTK stopwords
nltk.download('stopwords')

In [ ]:
# --- Configuration ---
PROJECT_PATH = '/content/drive/MyDrive/MarketGapMiner'
# (Replace YOUR_TOKEN_HERE with your actual token from https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTHTOKEN = "35IH5gYLd49YvDhxJDIQqub0vi2_6u79EiRgWLpQkmuZdH6Sx"

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- Create Project Folder ---
if not os.path.exists(PROJECT_PATH):
    os.makedirs(PROJECT_PATH)
    print(f"Folder '{PROJECT_PATH}' created!")
else:
    print(f"Folder '{PROJECT_PATH}' already exists.")

In [ ]:
def simulate_and_save_data(project_path: str) -> pd.DataFrame:
    """
    Generates a simulated dataset of product reviews and saves it to a CSV.
    """
    print("Simulating dataset...")
    data = {
        'product': np.random.choice(['Asana', 'ClickUp', 'Trello'], 500),
        'review_text': [
            "The billing is so confusing. I can never find my invoices.",
            "I love the UI, it's so clean.",
            "The interface is a total mess, I can't find anything.",
            "Why is the pricing so high? It's not worth the cost.",
            "Your customer support never answers my emails.",
            "This is the best tool ever!",
            "The mobile app is buggy and constantly crashes.",
            "I wish it had more integrations with my other tools.",
            "The integration with Salesforce is broken.",
            "The free tier is great, but the paid plans are a huge jump.",
            "I hate the new interface. Why did they change it?",
            "Customer support was amazing and solved my problem in 5 minutes.",
            "The billing department charged me twice! Took weeks to fix.",
            "Missing a key integration for my workflow.",
            "The mobile app needs a lot of work. It's almost unusable.",
            "Reporting is powerful, but the setup is a nightmare.",
            "The UI is beautiful but it's very slow to load.",
            "I don't understand the pricing model. It's too complex.",
            "No customer support on weekends. Very frustrating.",
            "The automation features are a game-changer."
        ] * 25  # Repeat list 25 times
    }

    df_raw = pd.DataFrame(data)

    # Save raw data
    output_path = f"{project_path}/simulated_reviews.csv"
    df_raw.to_csv(output_path, index=False)
    print(f"Simulated dataset created and saved to {output_path}")
    return df_raw

# --- Execute Data Simulation ---
df_raw = simulate_and_save_data(PROJECT_PATH)
df_raw.head()

In [ ]:
# Load NLP Models
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stop_words = set(stopwords.words('english'))
sentiment_analyzer = SentimentIntensityAnalyzer()

def clean_text(text: str) -> str:
    """
    Applies text cleaning pipeline:
    1. Lowercase
    2. Remove punctuation
    3. Remove numbers
    4. Lemmatize and remove stopwords
    """
    # 1. Lowercase
    text = text.lower()
    # 2. Remove punctuation
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    # 3. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 4. Lemmatize and remove stopwords
    doc = nlp(text)
    lemmatized_tokens = [
        token.lemma_ for token in doc
        if token.text not in stop_words and token.lemma_ not in stop_words
    ]

    # 5. Join tokens back to a string
    return " ".join(lemmatized_tokens)

def analyze_sentiment(df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies VADER sentiment analysis to the 'review_text' column.
    Uses the 'compound' score.
    """
    df['sentiment_score'] = df['review_text'].apply(
        lambda text: sentiment_analyzer.polarity_scores(text)['compound']
    )
    return df

#Test the cleaning function
test_sentence = "The billing is so confusing and terrible! I hate it 100%."
print(f"Original: {test_sentence}")
print(f"Cleaned: {clean_text(test_sentence)}")

#Apply processing to the main DataFrame
print("\nApplying sentiment scores and cleaning text...")
df_raw = analyze_sentiment(df_raw)
df_raw['cleaned_text'] = df_raw['review_text'].apply(clean_text)
print("Sentiment scores and cleaned text added.")
df_raw.head()

In [ ]:
def run_topic_modeling(documents: list) -> (BERTopic, list):
    """
    Fits a BERTopic model to a list of documents.
    """
    print("Starting BERTopic analysis...")
    topic_model = BERTopic(min_topic_size=3, verbose=True)
    topics, probabilities = topic_model.fit_transform(documents)
    print("BERTopic analysis complete.")
    return topic_model, topics

def calculate_gap_scores(df_pain: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the 'Gap_Score' based on frequency, sentiment, and competitor spread.
    """
    print("Calculating Gap Scores...")
    # We only want to score the topics we named
    df_scored = df_pain[df_pain['topic_name'] != "Other"]

    # Group by our new topic_name
    df_grouped = df_scored.groupby('topic_name')

    # Calculate the 3 parts of our formula
    df_gap_scores = df_grouped.agg(
        Frequency=('topic_name', 'size'),         # 1. Frequency of Complaint
        Avg_Sentiment=('sentiment_score', 'mean'), # 2. Average Negative Sentiment
        Competitor_Spread=('product', 'nunique')  # 3. Number of Competitors
    ).reset_index()

    # 2a. Make Avg_Sentiment positive (a high score should be "more negative")
    df_gap_scores['Avg_Sentiment'] = abs(df_gap_scores['Avg_Sentiment'])

    # Calculate the final Gap Score
    df_gap_scores['Gap_Score'] = (
        df_gap_scores['Frequency'] *
        df_gap_scores['Avg_Sentiment'] *
        df_gap_scores['Competitor_Spread']
    )

    # Sort to find the biggest opportunities
    df_gap_scores = df_gap_scores.sort_values(by='Gap_Score', ascending=False)

    return df_gap_scores

# Filter for negative reviews
df_pain = df_raw[df_raw['sentiment_score'] < -0.1].copy()
print(f"Original reviews: {len(df_raw)}")
print(f"Pain point reviews: {len(df_pain)}")

# Run Topic Modeling
documents = df_pain['cleaned_text'].tolist()
topic_model, topics = run_topic_modeling(documents)
df_pain['topic_id'] = topics

#  Manual Topic Mapping
print("--- Topic Model Keywords ---")
print(topic_model.get_topic_info())
print("----------------------------")

# This manual step is based on the keywords printed above
topic_map = {
    0: "Billing & Pricing", # Based on 'invoice', 'billing'
    1: "Interface & UI",    # Based on 'mess', 'interface'
    2: "Customer Support",  # Based on 'high', 'cost', 'pricing'
    3: "Integrations",      # Based on 'salesforce', 'integration'
    4: "Mobile App"         # Based on 'change', 'new', 'hate', 'interface' (likely refers to UI changes, mobile app is an assumption here)
}
df_pain['topic_name'] = df_pain['topic_id'].map(topic_map).fillna("Other")
print("\nTopic names mapped.")

# Calculate Scores & Save
df_gap_scores = calculate_gap_scores(df_pain)
print("\nGap Scores calculated:")
print(df_gap_scores)

#Save final data for the dashboard
df_gap_scores.to_csv(f"{PROJECT_PATH}/gap_scores.csv", index=False)
df_pain.to_csv(f"{PROJECT_PATH}/pain_reviews.csv", index=False)

print("\nFinal data saved to Google Drive.")
df_pain.head()

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px

# --- Config ---
st.set_page_config(layout="wide", page_title="MarketGap Miner")

# --- File Paths (adjust to your Drive folder) ---
PROJECT_PATH = '/content/drive/MyDrive/MarketGapMiner'
SCORES_FILE = f'{PROJECT_PATH}/gap_scores.csv'
REVIEWS_FILE = f'{PROJECT_PATH}/pain_reviews.csv'

#  Load Data
@st.cache_data
def load_data():
    """
    Loads the gap scores and review data from the CSV files.
    """
    try:
        df_scores = pd.read_csv(SCORES_FILE)
        df_reviews = pd.read_csv(REVIEWS_FILE)
        return df_scores, df_reviews
    except FileNotFoundError:
        st.error(f"ERROR: Data files not found. Make sure '{SCORES_FILE}' and '{REVIEWS_FILE}' exist.")
        return None, None

# Main App Logic
def main():
    df_scores, df_reviews = load_data()

    if df_scores is None or df_reviews is None:
        st.stop()

    # Header
    st.title("🚀 MarketGap Miner")
    st.markdown("This dashboard analyzes competitor reviews to find the biggest feature gaps and market opportunities.")

    # 1. Top Opportunities Chart
    st.header("🏆 Top Market Opportunities")
    st.markdown("This chart ranks complaint topics by the **Gap Score**. A high score means many people are very angry about this feature across multiple competitors.")

    fig = px.bar(
        df_scores,
        x='topic_name',
        y='Gap_Score',
        color='topic_name',
        title="Ranked Feature Gaps (Higher is a Bigger Opportunity)",
        labels={'topic_name': 'Complaint Topic', 'Gap_Score': 'Gap Score'}
    )
    fig.update_layout(xaxis_title=None, yaxis_title="Gap Score (Higher = More Pain)")
    st.plotly_chart(fig, use_container_width=True)

    # 2. Drill-Down "The Evidence"
    st.header("🔍 Drill-Down: The Evidence")
    st.markdown("Select a topic or competitor to read the *actual reviews* that generated the score.")

    # Filters
    col1, col2 = st.columns(2)
    with col1:
        # Get topics from the scores file so it's pre-filtered
        topic_list = ["All Topics"] + df_scores['topic_name'].unique().tolist()
        selected_topic = st.selectbox("Filter by Topic:", topic_list)

    with col2:
        product_list = ["All Products"] + df_reviews['product'].unique().tolist()
        selected_product = st.selectbox("Filter by Product:", product_list)

    # Filter the "evidence" dataframe
    df_filtered_reviews = df_reviews.copy()

    if selected_topic != "All Topics":
        df_filtered_reviews = df_filtered_reviews[df_filtered_reviews['topic_name'] == selected_topic]

    if selected_product != "All Products":
        df_filtered_reviews = df_filtered_reviews[df_filtered_reviews['product'] == selected_product]

    # Display the filtered reviews
    st.dataframe(
        df_filtered_reviews[['product', 'review_text', 'sentiment_score', 'topic_name']],
        height=400,
        use_container_width=True
    )
    st.success(f"Showing {len(df_filtered_reviews)} of {len(df_reviews)} total pain point reviews.")

if __name__ == "__main__":
    main()

In [ ]:
def run_streamlit_app():
    """
    Executes the Streamlit app using a shell command.
    """
    os.system('streamlit run app.py')

def launch_dashboard(authtoken: str):
    """
    Sets up ngrok, runs the Streamlit app in a thread,
    and prints the public URL.
    """
    # Set ngrok auth token
    ngrok.set_auth_token(authtoken)

    # Run streamlit in a separate thread
    print("Starting Streamlit app in background thread...")
    thread = threading.Thread(target=run_streamlit_app)
    thread.start()

    # Wait 5 seconds for the app to start
    time.sleep(5)

    # Connect ngrok to the streamlit port (8501)
    print("Connecting to ngrok...")
    public_url = ngrok.connect(8501)
    print(f"✅ Your app is live! Click here: {public_url}")

# --- Launch the app ---
launch_dashboard(NGROK_AUTHTOKEN)